# Reusing `importlib`, `nbconvert`, and `nbformat` to import notebooks

A lot of code is written in Jupyter notebooks, but little of it is reused. Currently, there is ambiguity in [how authors design their computational essays](https://blog.jupyter.org/we-analyzed-1-million-jupyter-notebooks-now-you-can-too-guest-post-8116a964b536). 

At deathbeds, we are making an effort to compose our notebooks so they can be reused as modules.  This means that our essays can be loaded into Python using the import system.  To do this we rely on importnb which is a package dedicated to importing notebooks. 

In this post we focus on the most basic steps to add notebooks to the input system.  Most demonstrations of custom imports use [a meta finder](http://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/Importing%20Notebooks.html4
), here we define a path hook. The path hook finder will look for notebooks matching a qualified notebook any where along the sys path.  Any notebook in a Python package becomes a potential module. 

    %reload_ext deathbeds.__The_simplest__import_notebook

In [1]:
    from importlib.machinery import SourceFileLoader, FileFinder
    from importlib.util import decode_source, cache_from_source
    import sys, inspect
    from nbconvert.exporters.python import PythonExporter
    from nbformat.v4 import reads
    Ø = __name__ == '__main__'

## The Notebook Source File Loader

The notebook loader is very similar to a Python loader, it just needs to convert the source json into valid Python. The standard source file loader has a [source_to_code](https://docs.python.org/3/library/importlib.html#importlib.abc.InspectLoader.source_to_code) method just for this purpose. To convert the notebook to python byte code we can use [nbconvert](https://github.com/jupyter/nbconvert).

In [2]:
    class NotebookLoader(SourceFileLoader):
        def source_to_code(self, data, path=None):
            return compile(PythonExporter().from_notebook_node(
                reads(decode_source(data))
            )[0], path, 'exec')

Using the native Source file loader means that the  imported module will be cached and is reloadable.

## Adding new loader details to an existing FileFinder

Creating a new meta finder is easy, adding a new path hook is not.  We are going to do some weird shit to discover the existing file finders for the existing Python imports.

In [3]:
    def find_finder():
        for i, hook in enumerate(sys.path_hooks):
            try: 
                if issubclass(inspect.getclosurevars(hook).nonlocals['cls'], 
                              FileFinder):return i, hook
            except: ...

In [4]:
    def new_loader(*details):
        i, finder = find_finder()
        sys.path_hooks[i] = FileFinder.path_hook(
             *details,*(
                 object for object in inspect.getclosurevars(find_finder()[1]).nonlocals['loader_details']
                 if not any(map('.ipynb'.__eq__, object[1]))))
        
        sys.path_importer_cache.clear()

It is essential to clear the path importer cache otherwise you will enter a waking nightmare while debugging.

## The IPython extension

In [5]:
    def load_ipython_extension(ip=None): 
        new_loader((NotebookLoader, ('.ipynb',)))
        get_ipython().ast_transformers = [] # This is a precaution for the demo
    Ø and load_ipython_extension()

## A Demonstration

In [6]:
    if Ø:
        import __style__
        assert isinstance(__style__.__loader__, NotebookLoader), "Some other thing is loaded."
        assert __import__('pathlib').Path(cache_from_source(__style__.__file__)).exists(), """The module is not cached"""

## Discussion

* This approach works, but it does not handle line numbers.  Without line numbers, it becomes difficult to debug a computational essay.
* Importing notebooks promotes an essay that will __restart and run all__, a condition essential for importing a Python module.
* Notebooks generally have more permissive file names.  The standard Python finder has a limited search strategy for most human named notebooks.  `importnb` attends a solution for a wildcard finders.  That is why the deathbeds posts - which start with numbers & contain hyphens - may be imported.

In [7]:
    if Ø: import disqus